# L1: Hierarchical Content Generation

## Creating a World

In [71]:
system_prompt = f"""
    Seu trabalho é ajudar a criar mundos de fantasia interessantes que os jogadores adorariam explorar.
   
    Instruções:
    Gere apenas texto simples, sem formatação.
    Use uma linguagem simples e clara, sem floreios.
    Cada descrição deve ter no máximo 3-5 frases.
    O conteúdo deve ser gerado em português
"""

In [72]:

world_prompt = f"""
Gere uma descrição criativa para um mundo de fantasia único.

Saída do conteúdo no formato:
Nome do Mundo: <NOME DO MUNDO>
Descrição do Mundo: <DESCRIÇÃO DO MUNDO>

Nome do Mundo:"""


**FUNCTIONS**

In [113]:
SCHEMA_KINGDOMS={"type":"object","properties":{"kingdoms":{"type":"array","items":{"type":"object","properties":{"name":{"type":"string"},"description":{"type":"string"},"world":{"type":"string"}},"required":["name","description","world"]}}},"required":["kingdoms"]}

SCHEMA_WORLD={"type":"object","properties":{"name":{"type":"string"},"description":{"type":"string"}}}

SCHEMA_TOWNS={"type":"object","properties":{"towns":{"type":"array","items":{"type":"object","properties":{"name":{"type":"string"},"description":{"type":"string"},"world":{"type":"string"},"kingdom":{"type":"string"}},"required":["name","description","world","kingdom"]}}},"required":["towns"]}



def generate_config(schema):
    return{
        "temperature": 1,
        "top_p": 0.95,
        "top_k": 40,
        "max_output_tokens": 8192,
        "response_mime_type": "application/json",
        "response_schema": schema
        
}

In [107]:
import google.generativeai as genai
from app_config import LLM_API_KEY,LLM_MODEL
import json


genai.configure(api_key=LLM_API_KEY)
client = genai.GenerativeModel(
            model_name=LLM_MODEL,
            system_instruction=system_prompt
            )

output = client.generate_content(world_prompt,generation_config=generate_config(SCHEMA_WORLD))

In [108]:
world =json.loads(output.text)
print(world)

{'description': 'Atheria é um mundo mágico onde a magia flui livremente através das plantas e animais, criando um ecossistema único e vibrante.  Grandes florestas flutuantes abrigam criaturas fantásticas e civilizações antigas.  Ruínas de uma poderosa civilização perdida escondem segredos poderosos e perigos ocultos.  Os habitantes de Atheria são seres mágicos, com a habilidade de se conectar com a natureza e manipular os elementos.  Exploradores e aventureiros são constantemente atraídos pelas paisagens de tirar o fôlego e a promessa de descobrimento.', 'name': 'Atheria'}


In [109]:
print(world['name'])
print(world['description'])

Atheria
Atheria é um mundo mágico onde a magia flui livremente através das plantas e animais, criando um ecossistema único e vibrante.  Grandes florestas flutuantes abrigam criaturas fantásticas e civilizações antigas.  Ruínas de uma poderosa civilização perdida escondem segredos poderosos e perigos ocultos.  Os habitantes de Atheria são seres mágicos, com a habilidade de se conectar com a natureza e manipular os elementos.  Exploradores e aventureiros são constantemente atraídos pelas paisagens de tirar o fôlego e a promessa de descobrimento.


## Generating Kingdoms

In [110]:
kingdom_prompt = f"""
Crie 3 reinos diferentes para um mundo de fantasia.
Para cada reino, gere uma descrição com base no mundo em que ele está.
Descreva líderes importantes, culturas e história do reino.

Saída do conteúdo no formato:
Nome do Reino 1: <NOME DO REINO>
Descrição do Reino 1: <DESCRIÇÃO DO REINO>
Nome do Reino 2: <NOME DO REINO>
Descrição do Reino 2: <DESCRIÇÃO DO REINO>
Nome do Reino 3: <NOME DO REINO>
Descrição do Reino 3: <DESCRIÇÃO DO REINO>

Nome do Mundo: {world['name']}
Descrição do Mundo: {world['description']}

Reino 1"""


In [111]:
output = client.generate_content(kingdom_prompt,generation_config=generate_config(SCHEMA_KINGDOMS))
output.text

'{"kingdoms": [{"description": "O Reino de Eldoria é conhecido por suas florestas antigas e sua cultura profundamente ligada à natureza. Seus habitantes, os Eldori, são seres pacíficos e sábios, com grande respeito pela magia natural. Liderados pela Rainha Lyra, uma curandeira poderosa, Eldoria prospera em harmonia com o ambiente, cultivando uma sociedade justa e equilibrada. Sua história é rica em lendas e tradições antigas, preservadas pelas gerações de druidas e xamãs.  Eldoria mantém relações amistosas com os reinos vizinhos, valorizando a diplomacia e a cooperação.", "name": "Eldoria", "world": "Atheria"}, {"description": "O Reino de Aerilon é uma nação costeira conhecida por sua poderosa marinha e sua cultura aventureira.  Os Aerilones são um povo corajoso e independente, com uma longa história de exploração e comércio. Liderados pelo Rei Theron, um estrategista brilhante, Aerilon domina a navegação e o comércio marítimo em Atheria. Sua história é marcada por grandes viagens e de

In [112]:
kingdoms = json.loads(output.text)
kingdoms

{'kingdoms': [{'description': 'O Reino de Eldoria é conhecido por suas florestas antigas e sua cultura profundamente ligada à natureza. Seus habitantes, os Eldori, são seres pacíficos e sábios, com grande respeito pela magia natural. Liderados pela Rainha Lyra, uma curandeira poderosa, Eldoria prospera em harmonia com o ambiente, cultivando uma sociedade justa e equilibrada. Sua história é rica em lendas e tradições antigas, preservadas pelas gerações de druidas e xamãs.  Eldoria mantém relações amistosas com os reinos vizinhos, valorizando a diplomacia e a cooperação.',
   'name': 'Eldoria',
   'world': 'Atheria'},
  {'description': 'O Reino de Aerilon é uma nação costeira conhecida por sua poderosa marinha e sua cultura aventureira.  Os Aerilones são um povo corajoso e independente, com uma longa história de exploração e comércio. Liderados pelo Rei Theron, um estrategista brilhante, Aerilon domina a navegação e o comércio marítimo em Atheria. Sua história é marcada por grandes viage

## Generating Towns

In [114]:
def get_town_prompt(world, kingdom):
    return f"""
            Crie 3 cidades diferentes para um reino e mundo de fantasia.
            Descreva a região em que está localizada, lugares importantes da cidade
            e sua história interessante.

            Saída do conteúdo no formato:
            Nome da Cidade 1: <NOME DA CIDADE>
            Descrição da Cidade 1: <DESCRIÇÃO DA CIDADE>
            Nome da Cidade 2: <NOME DA CIDADE>
            Descrição da Cidade 2: <DESCRIÇÃO DA CIDADE>
            Nome da Cidade 3: <NOME DA CIDADE>
            Descrição da Cidade 3: <DESCRIÇÃO DA CIDADE>

            Nome do Mundo: {world['name']}
            Descrição do Mundo: {world['description']}

            Nome do Reino: {kingdom['name']}
            Descrição do Reino: {kingdom['description']}

            Nome da Cidade 1:"""


In [128]:
def create_towns(world, kingdom):
    print(kingdom['name'])
    output = client.generate_content(get_town_prompt(world,kingdom),generation_config=generate_config(SCHEMA_TOWNS))
    output.text
    towns = json.loads(output.text)
    kingdom["towns"] = towns["towns"]

In [129]:
for kingdom in kingdoms['kingdoms']:
    create_towns(world, kingdom)  



Eldoria
Aerilon
Ignis


In [130]:
kingdoms

{'kingdoms': [{'description': 'O Reino de Eldoria é conhecido por suas florestas antigas e sua cultura profundamente ligada à natureza. Seus habitantes, os Eldori, são seres pacíficos e sábios, com grande respeito pela magia natural. Liderados pela Rainha Lyra, uma curandeira poderosa, Eldoria prospera em harmonia com o ambiente, cultivando uma sociedade justa e equilibrada. Sua história é rica em lendas e tradições antigas, preservadas pelas gerações de druidas e xamãs.  Eldoria mantém relações amistosas com os reinos vizinhos, valorizando a diplomacia e a cooperação.',
   'name': 'Eldoria',
   'world': 'Atheria',
   'towns': [{'description': 'Localizada no coração da Floresta de Eldoria, Lumina é uma cidade construída em torno de uma antiga árvore sagrada. Seus prédios são feitos de madeira e pedra, harmonizando com a natureza. O Grande Salão da Árvore é o centro da cidade, onde os anciãos se reúnem para tomar decisões. Lumina sempre foi um centro de aprendizado e conhecimento, abrig

In [142]:
kingdoms['kingdoms'][0]['towns'][0]

{'description': 'Localizada no coração da Floresta de Eldoria, Lumina é uma cidade construída em torno de uma antiga árvore sagrada. Seus prédios são feitos de madeira e pedra, harmonizando com a natureza. O Grande Salão da Árvore é o centro da cidade, onde os anciãos se reúnem para tomar decisões. Lumina sempre foi um centro de aprendizado e conhecimento, abrigando uma biblioteca repleta de antigos livros de magia natural.',
 'kingdom': 'Eldoria',
 'name': 'Lumina',
 'world': 'Atheria'}

In [143]:
town = kingdoms['kingdoms'][0]['towns'][0]
print(f'\nTown 1 Description: \
{town["description"]}')


Town 1 Description: Localizada no coração da Floresta de Eldoria, Lumina é uma cidade construída em torno de uma antiga árvore sagrada. Seus prédios são feitos de madeira e pedra, harmonizando com a natureza. O Grande Salão da Árvore é o centro da cidade, onde os anciãos se reúnem para tomar decisões. Lumina sempre foi um centro de aprendizado e conhecimento, abrigando uma biblioteca repleta de antigos livros de magia natural.


## Generating Non-Player Characters (NPC's)

In [ ]:
def get_npc_prompt(world, kingdom, town): 
    return f"""
    Create 3 different characters based on the world, kingdom \
    and town they're in. Describe the character's appearance and \
    profession, as well as their deeper pains and desires. \
    
    Output content in the form:
    Character 1 Name: <CHARACTER NAME>
    Character 1 Description: <CHARACTER DESCRIPTION>
    Character 2 Name: <CHARACTER NAME>
    Character 2 Description: <CHARACTER DESCRIPTION>
    Character 3 Name: <CHARACTER NAME>
    Character 3 Description: <CHARACTER DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description: {kingdom['description']}
    
    Town Name: {town['name']}
    Town Description: {town['description']}
    
    Character 1 Name:"""

In [ ]:
def create_npcs(world, kingdom, town):
    print(f'\nCreating characters for the town of: {town["name"]}...')
    output = client.chat.completions.create(
        model="meta-llama/Llama-3-70b-chat-hf",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_npc_prompt(world, kingdom, town)}
        ],
        temperature=1  #added to generate unique names
    )

    npcs_output = output.choices[0].message.content
    npcs = {}
    for output in npcs_output.split('\n\n'):
        npc_name = output.strip().split('\n')[0]\
        .split('Name: ')[1].strip()
        print(f'- "{npc_name}" created')
        
        npc_description = output.strip().split('\n')[1\
        ].split('Description: ')[1].strip()
        
        npc = {
        "name": npc_name,
        "description": npc_description,
        "world": world['name'],
        "kingdom": kingdom['name'],
        "town": town['name']
        }
        npcs[npc_name] = npc
    town["npcs"] = npcs

In [ ]:
for kingdom in kingdoms.values():
    for town in kingdom['towns'].values():
        create_npcs(world, kingdom, town)
  # For now we'll only generate npcs for one kingdom
    break

In [ ]:
npc = list(town['npcs'].values())[0]

print(f'\nNPC 1 in {town["name"]}, \
{kingdom["name"]}:\n{npc["description"]}')

## Save the World
>Note: You will save your world state to a file different than the one shown in the video to allow future lessons to be consistent with the video. If later wish to build your own worlds, you will want to load your file rather than the saved file.

In [ ]:
import json

def save_world(world, filename):
    with open(filename, 'w') as f:
        json.dump(world, f)

def load_world(filename):
    with open(filename, 'r') as f:
        return json.load(f)

#save_world(world, '../shared_data/Kyropeia.json')
save_world(world, '../shared_data/YourWorld_L1.json') #save to your version